In [3]:
pip install pywebio


     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
     --------------------- ------------------ 0.8/1.4 MB 2.8 MB/s eta 0:00:01
     ---------------------------- ----------- 1.0/1.4 MB 2.3 MB/s eta 0:00:01
     ---------------------------------------- 1.4/1.4 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pywebio: filename=pywebio-1.8.4-py3-none-any.whl size=1453399 sha256=a3e32181b64d3f6298d8f2106d23ca2450eb206ef5d7fbd8d2f280d012ef45c3
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\42\20\82\b3faa9b76112db87ad8c0f8a3d54341535d26ffea1722e0899
Successfully built pywebio


In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from pywebio import start_server
from pywebio.input import file_upload, select
from pywebio.output import *
from pywebio.session import run_async
import io
import base64
from datetime import datetime

class BankTransactionAnalyzer:
    def __init__(self):
        self.db_path = "bank_transactions_data_2.db"
        self.initialize_database()
    
    def initialize_database(self):
        """Initialize the SQLite database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS transactions (
            TransactionID TEXT PRIMARY KEY,
            AccountID TEXT,
            TransactionAmount REAL,
            TransactionDate TEXT,
            TransactionType TEXT,
            Location TEXT,
            DeviceID TEXT,
            IPAddress TEXT,
            MerchantID TEXT,
            Channel TEXT,
            CustomerAge INTEGER,
            CustomerOccupation TEXT,
            TransactionDuration INTEGER,
            LoginAttempts INTEGER,
            AccountBalance REAL,
            PreviousTransactionDate TEXT)''')
        conn.commit()
        conn.close()
    
    def upload_file(self):
        """Handle file upload and display preview"""
        file = file_upload("Upload Bank Transactions CSV", accept=".csv")
        if not file:
            return
        
        try:
            # Read and process file
            df = pd.read_csv(io.BytesIO(file['content']))
            
            # Clean column names by removing any extra spaces
            df.columns = [col.strip() for col in df.columns]
            
            # Convert date columns to datetime
            df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
            df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'])
            
            # Save to database
            conn = sqlite3.connect(self.db_path)
            df.to_sql("transactions", conn, if_exists="replace", index=False)
            conn.close()
            
            put_success("Transaction data uploaded successfully!")
            with use_scope('upload-content', clear=True):
                put_table(df.head(10).to_dict('records'))
        except Exception as e:
            toast(f"Error: {str(e)}", color='error')
    
    def get_data_from_db(self, query):
        """Execute SQL query and return DataFrame"""
        conn = sqlite3.connect(self.db_path)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    
    def transaction_analysis(self):
        """Show transaction amount analysis"""
        with use_scope('transaction-analysis', clear=True):
            put_markdown("## Transaction Analysis")
            
            analysis_type = select("Select analysis type", [
                'Top Transactions by Amount',
                'Transaction Type Distribution',
                'Daily Transaction Trends',
                'Transaction Amount by Location'
            ])
            
            if analysis_type == 'Top Transactions by Amount':
                df = self.get_data_from_db("""
                    SELECT TransactionID, AccountID, TransactionAmount, Location, TransactionType 
                    FROM transactions 
                    ORDER BY TransactionAmount DESC 
                    LIMIT 10""")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.barplot(x='TransactionID', y='TransactionAmount', 
                           data=df, palette='viridis', ax=ax)
                ax.set_title('Top 10 Transactions by Amount')
                ax.set_ylabel('Amount ($)')
                ax.set_xlabel('Transaction ID')
                plt.xticks(rotation=45)
                self.display_plot(fig)
                put_table(df.to_dict('records'))
                
            elif analysis_type == 'Transaction Type Distribution':
                df = self.get_data_from_db("""
                    SELECT TransactionType, COUNT(*) as Count, 
                    AVG(TransactionAmount) as AvgAmount 
                    FROM transactions 
                    GROUP BY TransactionType""")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
                
                # Transaction count by type
                sns.barplot(x='TransactionType', y='Count', 
                           data=df, palette='coolwarm', ax=ax1)
                ax1.set_title('Transaction Count by Type')
                ax1.set_ylabel('Number of Transactions')
                ax1.set_xlabel('Transaction Type')
                
                # Average amount by type
                sns.barplot(x='TransactionType', y='AvgAmount', 
                           data=df, palette='plasma', ax=ax2)
                ax2.set_title('Average Amount by Transaction Type')
                ax2.set_ylabel('Average Amount ($)')
                ax2.set_xlabel('Transaction Type')
                
                plt.tight_layout()
                self.display_plot(fig)
                put_table(df.to_dict('records'))
                
            elif analysis_type == 'Daily Transaction Trends':
                df = self.get_data_from_db("""
                    SELECT date(TransactionDate) as Date, 
                    SUM(TransactionAmount) as TotalAmount,
                    COUNT(*) as TransactionCount
                    FROM transactions 
                    GROUP BY date(TransactionDate)""")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                
                df['Date'] = pd.to_datetime(df['Date'])
                
                fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
                
                # Total amount over time
                sns.lineplot(x='Date', y='TotalAmount', 
                            data=df, marker='o', ax=ax1)
                ax1.set_title('Daily Transaction Amount Trends')
                ax1.set_ylabel('Total Amount ($)')
                
                # Transaction count over time
                sns.lineplot(x='Date', y='TransactionCount', 
                            data=df, marker='o', ax=ax2)
                ax2.set_title('Daily Transaction Count Trends')
                ax2.set_ylabel('Number of Transactions')
                
                plt.xticks(rotation=45)
                plt.tight_layout()
                self.display_plot(fig)
                put_table(df.to_dict('records'))
                
            elif analysis_type == 'Transaction Amount by Location':
                df = self.get_data_from_db("""
                    SELECT Location, 
                    AVG(TransactionAmount) as AvgAmount, 
                    COUNT(*) as TransactionCount 
                    FROM transactions 
                    GROUP BY Location
                    ORDER BY TransactionCount DESC
                    LIMIT 10""")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
                
                # Average transaction by location
                sns.barplot(x='Location', y='AvgAmount', 
                           data=df.sort_values('AvgAmount', ascending=False), 
                           palette='coolwarm', ax=ax1)
                ax1.set_title('Average Transaction Amount by Location')
                ax1.set_ylabel('Average Amount ($)')
                ax1.set_xlabel('Location')
                plt.sca(ax1)
                plt.xticks(rotation=45)
                
                # Transaction count by location
                sns.barplot(x='Location', y='TransactionCount', 
                           data=df.sort_values('TransactionCount', ascending=False), 
                           palette='viridis', ax=ax2)
                ax2.set_title('Transaction Count by Location')
                ax2.set_ylabel('Number of Transactions')
                ax2.set_xlabel('Location')
                plt.sca(ax2)
                plt.xticks(rotation=45)
                
                plt.tight_layout()
                self.display_plot(fig)
                put_table(df.to_dict('records'))
    
    def customer_analysis(self):
        """Analyze customer behavior"""
        with use_scope('customer-analysis', clear=True):
            put_markdown("## Customer Analysis")
            
            analysis_type = select("Select analysis type", [
                'Top Customers by Transaction Volume',
                'Customer Age Analysis',
                'Occupation-Based Analysis',
                'Account Balance Distribution'
            ])
            
            if analysis_type == 'Top Customers by Transaction Volume':
                df = self.get_data_from_db("""
                    SELECT AccountID, COUNT(*) as TransactionCount, 
                    SUM(TransactionAmount) as TotalAmount,
                    AVG(AccountBalance) as AvgBalance
                    FROM transactions 
                    GROUP BY AccountID 
                    ORDER BY TotalAmount DESC 
                    LIMIT 10""")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
                
                # Total amount by customer
                sns.barplot(x='AccountID', y='TotalAmount', 
                           data=df, palette='magma', ax=ax1)
                ax1.set_title('Total Transaction Amount by Customer')
                ax1.set_ylabel('Total Amount ($)')
                ax1.set_xlabel('Account ID')
                plt.sca(ax1)
                plt.xticks(rotation=45)
                
                # Transaction count by customer
                sns.barplot(x='AccountID', y='TransactionCount', 
                           data=df, palette='plasma', ax=ax2)
                ax2.set_title('Transaction Count by Customer')
                ax2.set_ylabel('Number of Transactions')
                ax2.set_xlabel('Account ID')
                plt.sca(ax2)
                plt.xticks(rotation=45)
                
                plt.tight_layout()
                self.display_plot(fig)
                put_table(df.to_dict('records'))
                
            elif analysis_type == 'Customer Age Analysis':
                df = self.get_data_from_db("SELECT CustomerAge FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                
                # Categorize into age groups
                bins = [0, 18, 25, 35, 50, 65, 100]
                labels = ['<18', '18-24', '25-34', '35-49', '50-64', '65+']
                df['AgeGroup'] = pd.cut(df['CustomerAge'], bins=bins, labels=labels)
                
                age_counts = df['AgeGroup'].value_counts().sort_index().reset_index()
                age_counts.columns = ['AgeGroup', 'Count']
                
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.barplot(x='AgeGroup', y='Count', data=age_counts, 
                            palette='coolwarm', ax=ax)
                ax.set_title('Customer Distribution by Age Group')
                ax.set_xlabel('Age Group')
                ax.set_ylabel('Number of Customers')
                self.display_plot(fig)
                put_table(age_counts.to_dict('records'))
                
            elif analysis_type == 'Occupation-Based Analysis':
                df = self.get_data_from_db("""
                    SELECT CustomerOccupation, 
                    AVG(TransactionAmount) as AvgAmount,
                    COUNT(*) as TransactionCount,
                    AVG(AccountBalance) as AvgBalance
                    FROM transactions 
                    GROUP BY CustomerOccupation
                    ORDER BY TransactionCount DESC
                    LIMIT 10""")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
                
                # Average transaction amount by occupation
                sns.barplot(x='CustomerOccupation', y='AvgAmount', 
                           data=df.sort_values('AvgAmount', ascending=False), 
                           palette='viridis', ax=ax1)
                ax1.set_title('Average Transaction Amount by Occupation')
                ax1.set_ylabel('Average Amount ($)')
                ax1.set_xlabel('Occupation')
                plt.sca(ax1)
                plt.xticks(rotation=45)
                
                # Transaction count by occupation
                sns.barplot(x='CustomerOccupation', y='TransactionCount', 
                           data=df.sort_values('TransactionCount', ascending=False), 
                           palette='plasma', ax=ax2)
                ax2.set_title('Transaction Count by Occupation')
                ax2.set_ylabel('Number of Transactions')
                ax2.set_xlabel('Occupation')
                plt.sca(ax2)
                plt.xticks(rotation=45)
                
                plt.tight_layout()
                self.display_plot(fig)
                put_table(df.to_dict('records'))
                
            elif analysis_type == 'Account Balance Distribution':
                df = self.get_data_from_db("SELECT AccountBalance FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.histplot(df['AccountBalance'], bins=20, kde=True, ax=ax)
                ax.set_title('Customer Account Balance Distribution')
                ax.set_xlabel('Account Balance ($)')
                ax.set_ylabel('Number of Customers')
                self.display_plot(fig)
    
    def channel_analysis(self):
        """Analyze transaction channels"""
        with use_scope('channel-analysis', clear=True):
            put_markdown("## Channel Analysis")
            
            analysis_type = select("Select analysis type", [
                'Transaction Channel Distribution',
                'Channel vs Amount',
                'Channel Usage by Age Group',
                'Channel Security Analysis'
            ])
            
            if analysis_type == 'Transaction Channel Distribution':
                df = self.get_data_from_db("""
                    SELECT Channel, COUNT(*) as Count,
                    AVG(TransactionAmount) as AvgAmount
                    FROM transactions
                    GROUP BY Channel""")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
                
                # Transaction count by channel
                sns.barplot(x='Channel', y='Count', 
                           data=df.sort_values('Count', ascending=False), 
                           palette='coolwarm', ax=ax1)
                ax1.set_title('Transaction Count by Channel')
                ax1.set_ylabel('Number of Transactions')
                ax1.set_xlabel('Channel')
                
                # Average amount by channel
                sns.barplot(x='Channel', y='AvgAmount', 
                           data=df.sort_values('AvgAmount', ascending=False), 
                           palette='viridis', ax=ax2)
                ax2.set_title('Average Amount by Channel')
                ax2.set_ylabel('Average Amount ($)')
                ax2.set_xlabel('Channel')
                
                plt.tight_layout()
                self.display_plot(fig)
                put_table(df.to_dict('records'))
                
            elif analysis_type == 'Channel vs Amount':
                df = self.get_data_from_db("SELECT Channel, TransactionAmount FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.boxplot(x='Channel', y='TransactionAmount', 
                           data=df, palette='coolwarm', ax=ax)
                ax.set_title('Transaction Amount Distribution by Channel')
                ax.set_ylabel('Amount ($)')
                ax.set_xlabel('Channel')
                plt.xticks(rotation=45)
                self.display_plot(fig)
                
            elif analysis_type == 'Channel Usage by Age Group':
                df = self.get_data_from_db("SELECT Channel, CustomerAge FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                
                # Categorize into age groups
                bins = [0, 18, 25, 35, 50, 65, 100]
                labels = ['<18', '18-24', '25-34', '35-49', '50-64', '65+']
                df['AgeGroup'] = pd.cut(df['CustomerAge'], bins=bins, labels=labels)
                
                channel_age = df.groupby(['Channel', 'AgeGroup']).size().reset_index()
                channel_age.columns = ['Channel', 'AgeGroup', 'Count']
                
                fig, ax = plt.subplots(figsize=(12, 6))
                sns.barplot(x='Channel', y='Count', hue='AgeGroup', 
                           data=channel_age, palette='viridis', ax=ax)
                ax.set_title('Channel Usage by Age Group')
                ax.set_ylabel('Number of Transactions')
                ax.set_xlabel('Channel')
                plt.xticks(rotation=45)
                plt.legend(title='Age Group')
                self.display_plot(fig)
                put_table(channel_age.to_dict('records'))
                
            elif analysis_type == 'Channel Security Analysis':
                df = self.get_data_from_db("SELECT Channel, LoginAttempts FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                    
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.boxplot(x='Channel', y='LoginAttempts', 
                           data=df, palette='coolwarm', ax=ax)
                ax.set_title('Login Attempts by Channel')
                ax.set_ylabel('Number of Login Attempts')
                ax.set_xlabel('Channel')
                plt.xticks(rotation=45)
                self.display_plot(fig)
    
    def time_analysis(self):
        """Analyze transaction patterns by time"""
        with use_scope('time-analysis', clear=True):
            put_markdown("## Time Analysis")
            
            analysis_type = select("Select analysis type", [
                'Hourly Transaction Patterns',
                'Weekday vs Weekend Analysis',
                'Transaction Time Distribution',
                'Time vs Amount Correlation'
            ])
            
            if analysis_type == 'Hourly Transaction Patterns':
                df = self.get_data_from_db("SELECT TransactionDate FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                
                # Extract hour from datetime
                df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
                df['Hour'] = df['TransactionDate'].dt.hour
                hourly_counts = df['Hour'].value_counts().sort_index().reset_index()
                hourly_counts.columns = ['Hour', 'Count']
                
                fig, ax = plt.subplots(figsize=(12, 6))
                sns.lineplot(x='Hour', y='Count', data=hourly_counts, 
                             marker='o', ax=ax)
                ax.set_title('Transaction Count by Hour of Day')
                ax.set_xlabel('Hour of Day (24-hour format)')
                ax.set_ylabel('Number of Transactions')
                ax.set_xticks(range(0, 24))
                self.display_plot(fig)
                put_table(hourly_counts.to_dict('records'))
                
            elif analysis_type == 'Weekday vs Weekend Analysis':
                df = self.get_data_from_db("SELECT TransactionDate, TransactionAmount FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                
                df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
                df['DayOfWeek'] = df['TransactionDate'].dt.day_name()
                df['IsWeekend'] = df['TransactionDate'].dt.dayofweek >= 5
                
                weekday_df = df.groupby('IsWeekend').agg({
                    'TransactionAmount': ['sum', 'mean'],
                    'TransactionDate': 'count'
                }).reset_index()
                
                weekday_df.columns = ['IsWeekend', 'TotalAmount', 'AvgAmount', 'Count']
                weekday_df['IsWeekend'] = weekday_df['IsWeekend'].map({True: 'Weekend', False: 'Weekday'})
                
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
                
                # Total amount
                sns.barplot(x='IsWeekend', y='TotalAmount', 
                           data=weekday_df, palette='coolwarm', ax=ax1)
                ax1.set_title('Total Transaction Amount')
                ax1.set_xlabel('')
                ax1.set_ylabel('Total Amount ($)')
                
                # Transaction count
                sns.barplot(x='IsWeekend', y='Count', 
                           data=weekday_df, palette='viridis', ax=ax2)
                ax2.set_title('Transaction Count')
                ax2.set_xlabel('')
                ax2.set_ylabel('Number of Transactions')
                
                plt.tight_layout()
                self.display_plot(fig)
                put_table(weekday_df.to_dict('records'))
                
            elif analysis_type == 'Transaction Time Distribution':
                df = self.get_data_from_db("SELECT TransactionDate FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                
                df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
                df['Hour'] = df['TransactionDate'].dt.hour
                
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.histplot(df['Hour'], bins=24, kde=True, ax=ax)
                ax.set_title('Transaction Time Distribution')
                ax.set_xlabel('Hour of Day')
                ax.set_ylabel('Number of Transactions')
                ax.set_xticks(range(0, 24))
                self.display_plot(fig)
                
            elif analysis_type == 'Time vs Amount Correlation':
                df = self.get_data_from_db("SELECT TransactionDate, TransactionAmount FROM transactions")
                
                if df.empty:
                    toast("No data available", color='error')
                    return
                
                df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
                df['Hour'] = df['TransactionDate'].dt.hour
                
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.scatterplot(x='Hour', y='TransactionAmount', 
                               data=df, alpha=0.6, ax=ax)
                ax.set_title('Transaction Amount by Time of Day')
                ax.set_xlabel('Hour of Day')
                ax.set_ylabel('Transaction Amount ($)')
                ax.set_xticks(range(0, 24))
                self.display_plot(fig)
    
    def display_plot(self, fig):
        """Display matplotlib figure as image"""
        img = io.BytesIO()
        plt.savefig(img, format='png', bbox_inches='tight', dpi=100)
        img.seek(0)
        img_data = base64.b64encode(img.read()).decode('utf-8')
        plt.close(fig)
        put_image(f'data:image/png;base64,{img_data}')
    
    def main(self):
        """Main application interface"""
        put_html("""
<style>
    :root {
        --primary-light: #a8c0ff;
        --primary-main: #6a8bf5;
        --primary-dark: #3a56b4;
        --text-primary: #4a5568;
        --text-secondary: #718096;
        --background-light: #f8fafc;
        --paper: rgba(255, 255, 255, 0.92);
        --success: #68d391;
        --warning: #f6ad55;
        --shadow-sm: 0 1px 3px rgba(0, 0, 0, 0.05);
        --shadow-md: 0 4px 12px rgba(0, 0, 0, 0.08);
    }
    
    @import url('https://fonts.googleapis.com/css2?family=Manrope:wght@300;400;500;600;700&display=swap');
    
    body {
        background: 
            linear-gradient(rgba(248, 250, 252, 0.92), rgba(248, 250, 252, 0.92)),
            url('https://images.unsplash.com/photo-1551288049-bebda4e38f71?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=2070&q=80') fixed;
        background-size: cover;
        color: var(--text-primary);
        font-family: 'Manrope', -apple-system, BlinkMacSystemFont, sans-serif;
        margin: 0;
        padding: 20px;
        min-height: 100vh;
        line-height: 1.6;
    }
    
    .pywebio-scope {
        background: var(--paper);
        padding: 28px;
        border-radius: 14px;
        margin-bottom: 24px;
        box-shadow: var(--shadow-md);
        border: 1px solid rgba(0, 0, 0, 0.04);
        transition: all 0.25s cubic-bezier(0.4, 0, 0.2, 1);
    }
    
    .pywebio-scope:hover {
        transform: translateY(-2px);
        box-shadow: 0 6px 16px rgba(0, 0, 0, 0.1);
    }
    
    .tab-button {
        transition: all 0.25s ease !important;
        border-radius: 10px !important;
        margin: 0 8px !important;
        padding: 10px 22px !important;
        background: rgba(255, 255, 255, 0.7) !important;
        color: var(--text-primary) !important;
        border: 1px solid rgba(0, 0, 0, 0.04) !important;
        font-weight: 500 !important;
        letter-spacing: 0.3px;
    }
    
    .tab-button.active {
        background: linear-gradient(135deg, var(--primary-main), var(--primary-dark)) !important;
        color: white !important;
        box-shadow: 0 3px 8px rgba(106, 139, 245, 0.2) !important;
    }
    
    .tab-button:hover:not(.active) {
        background: rgba(255, 255, 255, 0.9) !important;
    }
    
    h1 {
        color: var(--primary-dark);
        text-align: center;
        font-size: 2.4rem;
        margin: 10px 0 40px;
        font-weight: 700;
        letter-spacing: -0.5px;
        position: relative;
        padding-bottom: 12px;
    }
    
    h1:after {
        content: '';
        position: absolute;
        width: 80px;
        height: 4px;
        bottom: 0;
        left: 50%;
        transform: translateX(-50%);
        background: linear-gradient(90deg, var(--primary-light), var(--primary-main));
        border-radius: 2px;
    }
    
    button {
        background: linear-gradient(135deg, var(--primary-main), var(--primary-dark)) !important;
        color: white !important;
        border: none !important;
        padding: 12px 26px !important;
        border-radius: 10px !important;
        cursor: pointer !important;
        transition: all 0.25s ease !important;
        font-weight: 600 !important;
        box-shadow: var(--shadow-sm) !important;
        letter-spacing: 0.5px;
        position: relative;
        overflow: hidden;
    }
    
    button:hover {
        transform: translateY(-2px) !important;
        box-shadow: 0 6px 12px rgba(106, 139, 245, 0.25) !important;
    }
    
    button:active {
        transform: translateY(0) !important;
    }
    
    button:after {
        content: '';
        position: absolute;
        top: 0;
        left: -100%;
        width: 100%;
        height: 100%;
        background: linear-gradient(90deg, transparent, rgba(255, 255, 255, 0.2), transparent);
        transition: 0.5s;
    }
    
    button:hover:after {
        left: 100%;
    }
    
    /* Tab navigation */
    .tab-nav {
        background: var(--paper) !important;
        border-radius: 14px !important;
        padding: 10px !important;
        box-shadow: var(--shadow-md) !important;
        margin-bottom: 30px !important;
        border: 1px solid rgba(0, 0, 0, 0.04);
    }
    
    /* Table styling */
    table {
        border-radius: 12px !important;
        overflow: hidden !important;
        box-shadow: var(--shadow-sm) !important;
        background: white !important;
        border-collapse: separate;
        border-spacing: 0;
    }
    
    th {
        background: linear-gradient(135deg, var(--primary-main), var(--primary-dark)) !important;
        color: white !important;
        padding: 16px !important;
        font-weight: 600;
    }
    
    td {
        padding: 14px 16px !important;
        border-bottom: 1px solid rgba(0, 0, 0, 0.03) !important;
        color: var(--text-primary);
    }
    
    tr:last-child td {
        border-bottom: none !important;
    }
    
    tr:hover td {
        background: rgba(168, 192, 255, 0.05) !important;
    }
    
    /* Custom scrollbar */
    ::-webkit-scrollbar {
        width: 8px;
        height: 8px;
    }
    
    ::-webkit-scrollbar-track {
        background: rgba(0, 0, 0, 0.02);
        border-radius: 4px;
    }
    
    ::-webkit-scrollbar-thumb {
        background: rgba(106, 139, 245, 0.3);
        border-radius: 4px;
    }
    
    /* Notification toasts */
    .toast {
        border-radius: 10px !important;
        background: var(--primary-dark) !important;
        color: white !important;
        box-shadow: var(--shadow-md) !important;
        backdrop-filter: blur(10px);
    }
    
    /* Charts container */
    .plot-container {
        border-radius: 14px !important;
        overflow: hidden !important;
        box-shadow: var(--shadow-sm) !important;
        background: white !important;
        padding: 18px !important;
        border: 1px solid rgba(0, 0, 0, 0.04);
    }
    
    /* Input fields */
    input, select, textarea {
        border-radius: 10px !important;
        padding: 12px 16px !important;
        border: 1px solid rgba(0, 0, 0, 0.08) !important;
        background: white !important;
        transition: all 0.2s ease !important;
        color: var(--text-primary);
    }
    
    input:focus, select:focus, textarea:focus {
        border-color: var(--primary-main) !important;
        box-shadow: 0 0 0 3px rgba(106, 139, 245, 0.1) !important;
        outline: none !important;
    }
    
    /* Status indicators */
    .success-bg {
        background-color: var(--success) !important;
    }
    
    .warning-bg {
        background-color: var(--warning) !important;
    }
    
    /* Special text colors */
    .text-primary {
        color: var(--primary-main) !important;
    }
    
    .text-secondary {
        color: var(--text-secondary) !important;
    }
</style>
""")
        put_markdown("Bank Transaction Analyzer")
        
        put_tabs([
            {'title': 'Upload Data', 'content': [
                put_button("Upload Transaction Data", onclick=self.upload_file),
                put_scope('upload-content')
            ]},
            {'title': 'Transaction Analysis', 'content': [
                put_button("Analyze Transactions", onclick=self.transaction_analysis),
                put_scope('transaction-analysis')
            ]},
            {'title': ' Customer Analysis', 'content': [
                put_button("Analyze Customer Behavior", onclick=self.customer_analysis),
                put_scope('customer-analysis')
            ]},
            {'title': ' Channel Analysis', 'content': [
                put_button("Analyze Transaction Channels", onclick=self.channel_analysis),
                put_scope('channel-analysis')
            ]},
            {'title': 'Time Analysis', 'content': [
                put_button("Analyze Transaction Timing", onclick=self.time_analysis),
                put_scope('time-analysis')
            ]}
        ])

def main():
    app = BankTransactionAnalyzer()
    app.main()

if __name__ == '__main__':
    start_server(main, port=2027, debug=True) 

Running on all addresses.
Use http://10.30.62.171:2027/ to access the application


AssertionError: 

C:\Users\Dell\AppData\Local\Temp\ipykernel_14236\2500895940.py:124: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

C:\Users\Dell\AppData\Local\Temp\ipykernel_14236\2500895940.py:131: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

C:\Users\Dell\AppData\Local\Temp\ipykernel_14236\2500895940.py:191: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

C:\Users\Dell\AppData\Local\Temp\ipykernel_14236\2500895940.py:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effec